In [123]:
import pandas as pd
from datetime import datetime as dt

In [124]:
def remove_small_fires(data: pd.DataFrame, scn_trck='scan', size=1):
    other = 'track'
    if scn_trck == 'track':
        other = 'scan'
    keep_data = []
    for index, row in data[scn_trck].items():
        keep = True
        if row <= size:
            keep = False
        keep_data.append(keep)
    data = data[keep_data].reset_index().drop(columns=['index', other])
    return data
    

In [125]:
def get_date_range(data, start_date, end_date):
    """
    input in the form of a string "yyyy-mm-dd"
    """
    start_date = strip_date(start_date)
    end_date = strip_date(end_date)
    dates = data['acq_date']
    start_index = 0
    end_index = 0
    for index, row in dates.items():
        if strip_date(row) >= start_date:
            start_index = index
            break
    for index, row in reversed(dates.items()):
        if strip_date(row) >= start_date:
            end_index = -index - 1
            break
    print(start_index, end_index)

def strip_date(date):
    return dt.strptime(date, '%Y-%m-%d')

In [153]:
def remove_dates(data, date):
    for index, row in data['acq_date'].items():
        if row[:4] == date:
            print(row[:4])
            return data.iloc[index:, :]

In [154]:
def get_day_time_location(data):
    return data[['acq_time', 'acq_date', 'latitude', 'longitude']]

In [158]:
def load_fire_data(files: [list, str]):
    if type(files) == str:
        data = pd.read_csv(files)
    elif type(files) == list:
        data = pd.DataFrame([])
        for file in files:
            data = data.append(pd.read_csv(file))
    else:
        raise ValueError('files must be a list or a string')
    data = remove_dates(data, '2017')
    data = data[data['type'] == 0.0]
    data = data[data['confidence'] >= 100]
    data = data[data['brightness'] >= np.percentile(data['brightness'], 10)]
    data = data.reset_index().drop(columns=['index', 'version', 'instrument', 'daynight', 'type', 'bright_t31', 'confidence', 'satellite', 'frp'])
    data = remove_small_fires(data, size=1)
    data = data.drop(columns='scan')
    return data

In [159]:
data = load_fire_data(['data/DL_FIRE_M6_81124/fire_archive_M6_81124.csv', 'data/DL_FIRE_M6_81124/fire_nrt_M6_81124.csv'])

2017


In [160]:
data

,acq_date,acq_time,brightness,latitude,longitude
0,2017-01-12,1953,372.2,55.7452,-121.6579
1,2017-01-15,1846,389.4,54.1834,-117.2727
2,2017-01-15,1846,422.3,54.1854,-117.2909
3,2017-01-18,1917,384.5,53.0244,-116.5640
4,2017-01-18,2104,374.5,53.1377,-115.8775
...,...,...,...,...,...
25703,2019-07-31,615,332.4,67.2373,-131.4482
25704,2019-07-31,838,331.8,50.6842,-99.1288
25705,2019-07-31,1938,393.8,67.2150,-131.7663
25706,2019-07-31,2133,386.8,67.2350,-131.3381


In [87]:
max(data['latitude']), max(data['longitude'])

(69.6655, -52.9539)

In [88]:
min(data['latitude']), min(data['longitude'])

(42.0463, -141.004)

In [89]:
data['acq_date']

0         2004-01-09
1         2004-01-11
2         2004-01-11
3         2004-01-11
4         2004-01-11
             ...    
176741    2019-07-31
176742    2019-07-31
176743    2019-07-31
176744    2019-07-31
176745    2019-07-31
Name: acq_date, Length: 176746, dtype: object